In [6]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

In [7]:
load_dotenv()

API_KEY = os.getenv("API_KEY")


In [10]:
def fetch_external_data():
    """
    Fetch external data from an API.
    
    Returns: 
        DataFrame
    """
    api_url = "https://api.example.com/market-prices"
    headers = {" Authorization":f"Bearer{API_KEY}"}

    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        return pd.DataFrame(response.json())
    except requests.exceptions.RequestException as e:
        print(f"Error fetching external data: {e}")
        return pd.DataFrame()

def generate_insights(data, external_data):
    """
    Generate business insights by comparing product data with external data.

    Args:
        data (DataFrame): The product data.
        external_data (DataFrame): The external market data.

    Returns:
        str: A string containing formatted insights.
    """
    # Merge product data with external data on product_name
    merged_data = pd.merge(data, external_data, on="product_name", how="left")

    # Calculate the price difference
    merged_data["price_difference"] = merged_data["our_price"] - merged_data["market_price"]

    # Generate insights
    insights = "# Business Insights\n\n"
    for _, row in merged_data.iterrows():
        if row["price_difference"] > 0:
            insights += f"- {row['product_name']} is priced higher than the market average by ${row['price_difference']:.2f}.\n"
        elif row["price_difference"] < 0:
            insights += f"- {row['product_name']} is priced lower than the market average by ${-row['price_difference']:.2f}.\n"
        else:
            insights += f"- {row['product_name']} is priced at the market average.\n"

    return insights